In [ ]:
from scanphyslog2bids.core import PhilipsPhysioLog
import nibabel as nib
import numpy as np

main_dir = '/Users/jheunis/Desktop/sample-data/sub-neufepmetest/sub-pilot/raw-data'
log_file = main_dir + '/SCANPHYSLOG20191011115018_motor1.log'
out_dir = main_dir + '/my_bids_data'  # where the BIDSified data should be saved
deriv_dir = main_dir + '/my_bids_data/physio'  # where some QC plots should be saved

# fmri_file is used to extract metadata, such as TR and number of volumes
fmri_file = main_dir + '/sub-pilot_task-motor_run-1_echo-2.nii' 
fmri_img = nib.load(fmri_file)
n_dyns = fmri_img.shape[-1]
tr = np.round(fmri_img.header['pixdim'][4], 3)

# Create PhilipsPhysioLog object with info
phlog = PhilipsPhysioLog(f=log_file, tr=tr, n_dyns=n_dyns, sf=500, manually_stopped=False)

# Load in data, do some preprocessing
phlog.load()

# Try to align physio data with scan data, using a particular method
# (either "vol_markers", "gradient_log", or "interpolation")
# phlog.align(trigger_method='vol_markers')  # load and find vol triggers
phlog.align(trigger_method='interpolate')  # load and find vol triggers

# Write out BIDS files
phlog.to_bids(out_dir)  # writes out .tsv.gz and .json files

# Optional: plot some QC graphs for alignment and actual traces
phlog.plot_alignment(out_dir=deriv_dir)  # plots alignment with gradient
phlog.plot_traces(out_dir=deriv_dir)  # plots cardiac/resp traces